In [90]:
import requests
from bs4 import BeautifulSoup as BS
import json
from tqdm import tqdm
from zipfile import ZipFile, ZIP_DEFLATED

In [3]:
proxy_url = '45.81.76.252'
proxy_port = '8000'

proxy = {
    'https': f'http://{proxy_url}:{proxy_port}'
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/111.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8'
}

In [5]:
params = {
    'searchString': '',
    'morphology': 'on',
    'search-filter': 'Дате обновления',
    'savedSearchSettingsIdHidden': '',
    'fz94': 'on',
    'fz223': 'on',
    'published': 'on',
    'regarded': 'on',
    'considered': 'on',
    'decisionOnTheComplaintTypeResult': '',
    'receiptDateStart': '',
    'receiptDateEnd': '',
    'updateDateFrom': '',
    'updateDateTo': '',
    'sortBy': 'UPDATE_DATE',
    'pageNumber': '',
    'sortDirection': 'false',
    'recordsPerPage': '_10',
    'showLotsInfoHidden': 'false',
}

In [6]:
url = """

https://zakupki.gov.ru/epz/complaint/search/search_eis.html

""".strip()
host = '/'.join(url.split('/', maxsplit=3)[:3])


In [7]:
def _parse_links_in_page(text: str) -> list[str]:
    # Получам ссылки с каждой страницы (всего 10)
    out_list = list()
    soup = BS(text, 'html.parser')
    blocks_list = soup.find('div', class_ = 'search-registry-entrys-block')
    blocks_list = blocks_list.find_all('div', class_ = 'search-registry-entry-block')

    for block in blocks_list:
        href = block.find('div', class_ = 'd-flex registry-entry__header-mid align-items-center')
        href = host + href.find('span').find('a').get('href')
        out_list.append(href)
    
    return out_list

def _parse_links_in_site() -> list[str]:
    # Проходим по страницам поиска из реестра жалоб
    # всего страниц 100
    # на каждой странице 10 жалоб
    # на выходе получаем список ссылок на страницу с жалобой
    out_list = list()
    
    with tqdm(total=100 ) as pbar:
        for page in range(1,101):
            params['pageNumber'] = f'{page}'
            response = requests.get(url=url,params=params , headers=headers, proxies=proxy)
            out_list.append(_parse_links_in_page(response.text))
            pbar.update(1)
    
    return sum(out_list, [])

In [8]:
links_list = _parse_links_in_site()

100%|██████████| 100/100 [01:06<00:00,  1.49it/s]


In [74]:
# Сохранение линков в json формате
#--------------------------------------------
# with open('data/links_list.json', 'w', encoding='utf-8') as file:
#     tmp = json.dumps(links_list, indent=4, ensure_ascii=False)
#     file.write(tmp)

In [ ]:
# Загрузка линков из json
#--------------------------------------------
# with open('data/links_list.json', 'r', encoding='utf-8') as file:
#     tmp = file.read()
#     links_list = json.loads(tmp)

In [32]:
links_list[1]

'https://zakupki.gov.ru/epz/complaint/card/complaint-information.html?id=2258395'

In [ ]:




# number the complaint - Номер жалобы
# date the complaint - Дата размещения жалобы

# Participant data - Данные участника -----------------
# Type of participant - Тип участника
# Name of participant - Имя участника
# INN of participant - Имя участника
# Date of registration - Дата постановки на учет в налоговом органе
# Location of participant - Место нахождения
# Phone of participant - телефон участника
# email - Адрес электронной почты

# Information about the subject of control - Информация о субъекте контроля ------------
# Subject of control - Субъект контроля
# Name of the organization - Наименование организации
# Postal address - Почтовый адрес
# Location of the subject - Место нахождения
# Phone of the subject - Номер телефона

In [94]:
def _parse_info_complaint(text: str):
    soup = BS(text, 'html.parser')
    complaint = soup.find('div', class_='sectionMainInfo borderRight col-9')

    number_complaint = complaint.find('div', class_='cardMainInfo__status').find('a').get_text()[2:]

    status_complaint = complaint.find('div', class_='cardMainInfo__status').find('span', class_ = 'cardMainInfo__state').get_text()

    comments_complaint = list()
    try:
        tmp = complaint.find('span', class_='cardMainInfo__title distancedText float-left').find('span').get_text().strip()
    except AttributeError:
        tmp = ''
    comments_complaint.append(tmp)
    try:
        tmp = complaint.find('span', class_='cardMainInfo__title distancedText float-left').find_next_sibling('span', class_='cardMainInfo__title').get_text().strip()
    except AttributeError:
        tmp = ''
    comments_complaint.append(tmp)

    card_common = soup.find('div', class_='card-common')
    date_complaint = card_common.find(string='Дата размещения жалобы').find_parent().find_next_sibling('div', class_="common-text__value").get_text().strip()
    print(number_complaint)


def _get_number(text: str) -> str:
    soup = BS(text, 'html.parser')
    complaint = soup.find('div', class_='sectionMainInfo borderRight col-9')
    number_complaint = complaint.find('div', class_='cardMainInfo__status').find('a').get_text()[2:]
    return number_complaint

In [92]:
# for url in links_list[:10]:
#     response = requests.get(url=url, headers=headers, proxies=proxy)
#     _parse_info_complaint(response.text)


def load_pages(links_list: list[str]) -> dict[str, str]:
    with tqdm(total=len(links_list)) as qbar:
        out_dict = dict()
        for link in links_list:
            response = requests.get(url=link, headers=headers, proxies=proxy)
            number = _get_number(response.text)
            out_dict[number] = response.text
            qbar.update(1)
        return out_dict

def save_pages(in_dict: dict[str, str]) -> None:
    with ZipFile('data/pages.zip', 'w', compression=ZIP_DEFLATED, compresslevel=1) as zip_file:
        for name, value in in_dict.items():
            zip_file.writestr(f'{name}.html', value)

In [96]:
pages_dict = load_pages(links_list=links_list)

100%|██████████| 1000/1000 [08:30<00:00,  1.96it/s]


In [97]:
save_pages(pages_dict)